In [30]:
import json

json_data = """
{
    "orderDate": "2024-09-03",
    "shipDate": "2024-10-31",
    "machinesAvailable": "12",
    "fabricDetails": [
        {
            "name": "BAG VOILE",
            "usage": {
                "value": "1.4",
                "unit": "meter"
            },
            "processes": [
                "DYING",
                "PRINTING"
            ],
            "colors": [
                {
                    "name": "Red",
                    "quantity": "2000"
                },
                {
                    "name": "Green",
                    "quantity": "5000"
                }
            ]
        },
        {
            "name": "SHEETING",
            "usage": {
                "value": "1.8",
                "unit": "meter"
            },
            "processes": [
                "EMBROIDERY",
                "SHIFFLY",
                "DYING"
            ],
            "colors": [
                {
                    "name": "Blue",
                    "quantity": "7000"
                }
            ]
        }
    ],
    "accessories": [
        {
            "name": "Button",
            "id": 1726139455693
        },
        {
            "name": "Zipper",
            "id": 1726139646344
        },
        {
            "name": "Hook",
            "id": 1726139648212
        },
        {
            "name": "Elastic",
            "id": 1726139650762
        }
    ],
    "trims": [
        {
            "name": "Label",
            "id": 1726139455693
        },
        {
            "name": "Tag",
            "id": 1726139657824
        },
        {
            "name": "Wash Care",
            "id": 1726139660580
        }
    ]
}
"""

python_dict = json.loads(json_data)

print(python_dict)


{'orderDate': '2024-09-03', 'shipDate': '2024-10-31', 'machinesAvailable': '12', 'fabricDetails': [{'name': 'BAG VOILE', 'usage': {'value': '1.4', 'unit': 'meter'}, 'processes': ['DYING', 'PRINTING'], 'colors': [{'name': 'Red', 'quantity': '2000'}, {'name': 'Green', 'quantity': '5000'}]}, {'name': 'SHEETING', 'usage': {'value': '1.8', 'unit': 'meter'}, 'processes': ['EMBROIDERY', 'SHIFFLY', 'DYING'], 'colors': [{'name': 'Blue', 'quantity': '7000'}]}], 'accessories': [{'name': 'Button', 'id': 1726139455693}, {'name': 'Zipper', 'id': 1726139646344}, {'name': 'Hook', 'id': 1726139648212}, {'name': 'Elastic', 'id': 1726139650762}], 'trims': [{'name': 'Label', 'id': 1726139455693}, {'name': 'Tag', 'id': 1726139657824}, {'name': 'Wash Care', 'id': 1726139660580}]}


In [31]:
# Days In Hand =Shipping  Days  - Order Date    --- Done
# Machines Required  =  (Total Order Quantity / Days in Stiching ) / Single Machine Capacity
# Order Quantity = Max ( total Quantity from different Fabrics)  --- Done 

In [32]:
from datetime import datetime

python_dict["shipDate"] = datetime.strptime(python_dict["shipDate"], '%Y-%m-%d')
python_dict["orderDate"] = datetime.strptime(python_dict["orderDate"], '%Y-%m-%d')

In [33]:
days_in_hand = (python_dict["shipDate"]-python_dict["orderDate"]).days 

In [34]:
max_order_quantity = 0
for fabric in python_dict["fabricDetails"]:
    sum_of_fabric_colors = 0
    for color_details in fabric["colors"]:
        sum_of_fabric_colors+=int(color_details["quantity"])
    if max_order_quantity<=sum_of_fabric_colors:
        max_order_quantity=sum_of_fabric_colors

max_order_quantity


7000

In [35]:
import pandas as pd
from datetime import timedelta
df_procurement_days = pd.read_excel("C:/Users/DKC/Desktop/Python Server Processing Code/Excel Files/Purchase.xlsx")

def get_procurement_days(Material, quantity):
    # Filter the DataFrame based on the fabric
    fabric_data = df_procurement_days[df_procurement_days['Material'] == Material]
    
    # Iterate through the rows to find the matching quantity range
    for _, row in fabric_data.iterrows():
        if row['Quantity Minimum'] <= quantity <= row['Quantity Maximum']:
            return row['Minimum days']
    
    return "Quantity out of range or fabric not found"

procurement_details = []

# Example usage
for material_details in python_dict["fabricDetails"]:
    material_name =  material_details["name"]
    total_fabric_quantity=0
    for color_details in material_details["colors"]:
        total_fabric_quantity+=int(color_details["quantity"])

    procurement_days = get_procurement_days(material_name,total_fabric_quantity)
    print(f"Procurement days for {total_fabric_quantity}kg of {material_name}: {procurement_days} days")

    procurement_details.append({
        "material_name": material_name,
        "Total Quantity": total_fabric_quantity,
        "Start Date": python_dict["orderDate"],
        "Minimum Days Required": procurement_days,
        "Expected Completion Date": python_dict["orderDate"]+timedelta(days=procurement_days)
    })

procurement_details

Procurement days for 7000kg of BAG VOILE: 7 days
Procurement days for 7000kg of SHEETING: 7 days


[{'material_name': 'BAG VOILE',
  'Total Quantity': 7000,
  'Start Date': datetime.datetime(2024, 9, 3, 0, 0),
  'Minimum Days Required': 7,
  'Expected Completion Date': datetime.datetime(2024, 9, 10, 0, 0)},
 {'material_name': 'SHEETING',
  'Total Quantity': 7000,
  'Start Date': datetime.datetime(2024, 9, 3, 0, 0),
  'Minimum Days Required': 7,
  'Expected Completion Date': datetime.datetime(2024, 9, 10, 0, 0)}]

In [36]:
#FOB STAGE

# {"name": "BAG VOILE",
#             "Parent_Category": "WOVEN"
#             "usage": {
#                 "value": "1.4",
#                 "unit": "meter"
#             },
#             "processes": [
#                 "Dyeing",
#                 "Printing"
#             ],
#             "colors": [
#                 {
#                     "name": "Red",
#                     "quantity": "2000"
#                 },
#                 {
#                     "name": "Green",
#                     "quantity": "5000"
#                 }
#             ]
#         }


#Handling First Case


df_fob_days = pd.read_excel("C:/Users/DKC/Desktop/Python Server Processing Code/Excel Files/fob.xlsx")
df_fob_days

,parent_category,process,minimum_days,maximum_days
0,WOVEN,DYING,7,10
1,WOVEN,PRINTING,10,20
2,WOVEN,SHIFFLY,1,2
3,WOVEN,EMBROIDERY,1,2
4,WOVEN,LACES,1,2
5,KNITTED,KNITTED,10,15


In [37]:


fabric_name = "BAG VOILE"
Parent_Category= "WOVEN"
process_fob = "DYING"

fob_minimum_days = df_fob_days[(df_fob_days["parent_category"]==Parent_Category) & (df_fob_days["process"]==process_fob)]["minimum_days"]

print(fob_minimum_days)


print(json_data)







0    7
Name: minimum_days, dtype: int64

{
    "orderDate": "2024-09-03",
    "shipDate": "2024-10-31",
    "machinesAvailable": "12",
    "fabricDetails": [
        {
            "name": "BAG VOILE",
            "usage": {
                "value": "1.4",
                "unit": "meter"
            },
            "processes": [
                "DYING",
                "PRINTING"
            ],
            "colors": [
                {
                    "name": "Red",
                    "quantity": "2000"
                },
                {
                    "name": "Green",
                    "quantity": "5000"
                }
            ]
        },
        {
            "name": "SHEETING",
            "usage": {
                "value": "1.8",
                "unit": "meter"
            },
            "processes": [
                "EMBROIDERY",
                "SHIFFLY",
                "DYING"
            ],
            "colors": [
                {
                    "n

In [38]:
for fabric_details in python_dict["fabricDetails"]:
    fabric_name = fabric_details["name"]
    Parent_Category= "WOVEN"
    for process_names in  fabric_details["processes"]:
        process_name = process_names
        fob_minimum_days = df_fob_days[(df_fob_days["parent_category"]==Parent_Category) & (df_fob_days["process"]==process_name)]["minimum_days"]
        print(int(fob_minimum_days.iloc[0]))
        





7
10
1
1
7


In [39]:


# Load the FOB days data from the Excel file
df_fob_days = pd.read_excel("C:/Users/DKC/Desktop/Python Server Processing Code/Excel Files/fob.xlsx")

# Sample input data (you would replace this with your actual data)
python_dict = {
    "fabricDetails": [
        {
            "name": "BAG VOILE",
            "parent_category": "WOVEN",
            "colors": [
                {"color": "Red", "quantity": 1500},
                {"color": "Blue", "quantity": 2000}
            ],
            "processes": ["DYING", "PRINT"]
        },
        # Add more fabric details as needed
    ]
}

# Initialize an empty list to store the results
result = []

# Iterate through each fabric detail in the input data
for fabric_details in python_dict["fabricDetails"]:
    fabric_name = fabric_details["name"]
    parent_category = fabric_details["parent_category"]
    for color_details in fabric_details["colors"]:
        color = color_details["color"]
        quantity = color_details["quantity"]
        for process_name in fabric_details["processes"]:
            # Get the minimum days for the FOB process
            fob_minimum_days = df_fob_days[
                (df_fob_days["parent_category"] == parent_category) & 
                (df_fob_days["process"] == process_name)
            ]["minimum_days"]
            
            if not fob_minimum_days.empty:
                fob_minimum_days = int(fob_minimum_days.iloc[0])
            else:
                fob_minimum_days = "N/A"  # or some default value or handling if not found
            
            # Append the details to the result list
            result.append({
                "fabric_name": fabric_name,
                "parent_category": parent_category,
                "color": color,
                "quantity": quantity,
                "process_name": process_name,
                "fob_minimum_days": fob_minimum_days
            })

# Print the result list
for entry in result:
    print(entry)

# Optionally, convert the result list to a DataFrame and save to an Excel file
df_result = pd.DataFrame(result)
df_result.to_excel("fob_process_details.xlsx", index=False)


{'fabric_name': 'BAG VOILE', 'parent_category': 'WOVEN', 'color': 'Red', 'quantity': 1500, 'process_name': 'DYING', 'fob_minimum_days': 7}
{'fabric_name': 'BAG VOILE', 'parent_category': 'WOVEN', 'color': 'Red', 'quantity': 1500, 'process_name': 'PRINT', 'fob_minimum_days': 'N/A'}
{'fabric_name': 'BAG VOILE', 'parent_category': 'WOVEN', 'color': 'Blue', 'quantity': 2000, 'process_name': 'DYING', 'fob_minimum_days': 7}
{'fabric_name': 'BAG VOILE', 'parent_category': 'WOVEN', 'color': 'Blue', 'quantity': 2000, 'process_name': 'PRINT', 'fob_minimum_days': 'N/A'}
